<font size="+3"><strong> Wrangling Data with MongoDB 🇰🇪</strong></font>

In [1]:
from pprint import PrettyPrinter

import pandas as pd
from pymongo import MongoClient

**Task 3.1.1:** Instantiate a `PrettyPrinter`, and assign it to the variable `pp`.


In [2]:
pp = PrettyPrinter(indent=2)

# Prepare Data

## Connect

**Task 3.1.2:** Create a client that connects to the database running at `localhost` on port `27017`.

In [3]:
client = MongoClient(host="localhost",port=27017)

## Explore

**Task 3.1.3:** Print a list of the databases available on `client`.


In [4]:
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960},
  {'empty': False, 'name': 'air-quality', 'sizeOnDisk': 5406720},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 110592},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 90112}]


**Task 3.1.4:** Assign the `"air-quality"` database to the variable `db`.

In [5]:
db = client["air-quality"]

**Task 3.1.5:** Use the [`list_collections`](https://pymongo.readthedocs.io/en/stable/api/pymongo/database.html?highlight=list_collections#pymongo.database.Database.list_collections) method to print a list of the collections available in `db`.

In [6]:
list(db.list_collections())[2]

{'name': 'nairobi',
 'type': 'timeseries',
 'options': {'timeseries': {'timeField': 'timestamp',
   'metaField': 'metadata',
   'granularity': 'seconds',
   'bucketMaxSpanSeconds': 3600}},
 'info': {'readOnly': False}}

In [7]:
for c in db.list_collections():
    print(c["name"])

lagos
system.buckets.lagos
nairobi
system.buckets.nairobi
dar-es-salaam
system.buckets.dar-es-salaam
system.views


**Task 3.1.6:** Assign the `"nairobi"` collection in `db` to the variable name `nairobi`.


In [8]:
nairobi = db["nairobi"]

**Task 3.1.7:** Use the [`count_documents`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.count_documents) method to see how many documents are in the `nairobi` collection.


In [9]:
nairobi.count_documents({})

435206

**Task 3.1.8:** Use the [`find_one`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find_one) method to retrieve one document from the `nairobi` collection, and assign it to the variable name `result`.

In [10]:
result = nairobi.find_one({})
pp.pprint(result)

{ 'P2': 34.43,
  '_id': ObjectId('64624fdf2d03b2bab2d399f2'),
  'metadata': { 'lat': -1.3,
                'lon': 36.785,
                'measurement': 'P2',
                'sensor_id': 57,
                'sensor_type': 'SDS011',
                'site': 29},
  'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)}


**Task 3.1.9:** Use the [`distinct`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.distinct) method to determine how many sensor sites are included in the `nairobi` collection.

In [11]:
nairobi.distinct("metadata.site")

[29, 6]

In [12]:
nairobi.distinct("metadata.measurement")

['humidity', 'P2', 'P1', 'temperature']

**Task 3.1.10:** Use the [`count_documents`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.count_documents) method to determine how many readings there are for each site in the `nairobi` collection.

In [13]:
print("Documents from site 6:", nairobi.count_documents({"metadata.site":6}))
print("Documents from site 29:",  nairobi.count_documents({"metadata.site":29}))

Documents from site 6: 173036
Documents from site 29: 262170


**Task 3.1.11:** Use the [`aggregate`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.aggregate) method to determine how many readings there are for each site in the `nairobi` collection.

In [14]:
result = nairobi.aggregate(
    [
        {"$group":{"_id":"$metadata.site","count":{"$count":{}}}}
    ]
)
pp.pprint(list(result))

[{'_id': 6, 'count': 173036}, {'_id': 29, 'count': 262170}]


**Task 3.1.12:** Use the [`distinct`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.distinct) method to determine how many types of measurements have been taken in the `nairobi` collection.

In [15]:
nairobi.distinct("metadata.measurement")

['humidity', 'P2', 'P1', 'temperature']

**Task 3.1.13:** Use the [`find`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find) method to retrieve the PM 2.5 readings from all sites. Be sure to limit your results to 3 records only.


In [16]:
result = nairobi.find({"metadata.measurement":"P2"}).limit(3)
pp.pprint(list(result))

[ { 'P2': 34.43,
    '_id': ObjectId('64624fdf2d03b2bab2d399f2'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)},
  { 'P2': 29.67,
    '_id': ObjectId('64624fdf2d03b2bab2d399f6'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': datetime.datetime(2018, 9, 1, 0, 2, 32, 885000)},
  { 'P2': 30.53,
    '_id': ObjectId('64624fdf2d03b2bab2d399fa'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': datetime.datetim

**Task 3.1.14:** Use the [`aggregate`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.aggregate) method to calculate how many readings there are for each type (`"humidity"`, `"temperature"`, `"P2"`, and `"P1"`) in site `6`.

In [17]:
result = nairobi.aggregate(
    [
        {"$match":{"metadata.site":6}},
        {"$group":{"_id":"$metadata.measurement","count":{"$count":{}}}}
    ]
)
pp.pprint(list(result))

[ {'_id': 'P1', 'count': 44624},
  {'_id': 'temperature', 'count': 41894},
  {'_id': 'humidity', 'count': 41894},
  {'_id': 'P2', 'count': 44624}]


**Task 3.1.15:** Use the [`aggregate`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.aggregate) method to calculate how many readings there are for each type (`"humidity"`, `"temperature"`, `"P2"`, and `"P1"`) in site `29`.

In [18]:
result = nairobi.aggregate(
    [
        {"$match":{"metadata.site":29}},
        {"$group":{"_id":"$metadata.measurement","count":{"$count":{}}}}
    ]
)
pp.pprint(list(result))

[ {'_id': 'P2', 'count': 65432},
  {'_id': 'humidity', 'count': 65653},
  {'_id': 'temperature', 'count': 65653},
  {'_id': 'P1', 'count': 65432}]


## Import

**Task 3.1.16:** Use the [`find`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find) method to retrieve the PM 2.5 readings from site `29`. Be sure to limit your results to 3 records only. Since we won't need the metadata for our model, use the `projection` argument to limit the results to the `"P2"` and `"timestamp"` keys only.

In [33]:
result = nairobi.find(
    {"metadata.site":29,"metadata.measurement":"P2"},
    projection={"P2":1,"timestamp":1,"_id":0}
)


**Task 3.1.17:** Read records from your `result` into the DataFrame `df`. Be sure to set the index to `"timestamp"`.

In [34]:
df = pd.DataFrame(result).set_index("timestamp")
df.head()

,P2
timestamp,
2018-09-01 00:00:02.472,34.43
2018-09-01 00:02:32.885,29.67
2018-09-01 00:05:03.941,30.53
2018-09-01 00:07:33.748,25.77
2018-09-01 00:10:04.374,22.80
